# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `--- text used to import the data from the Terminal -- 
----------------------------------------------------------------------------------------------------------------
Last login: Sat May  6 20:45:47 on ttys003
(base) Crisaldry@MacBook-Air Resources % conda activate PythonData
(PythonData) Crisaldry@MacBook-Air Resources % brew services start mongodb-community@6.0
==> Successfully started `mongodb-community` (label: homebrew.mxcl.mongodb-commu
(PythonData) Crisaldry@MacBook-Air Resources % mongoimport --db uk_food --collection establishments --file establishments.json

2023-05-06T20:48:00.151-0400	connected to: mongodb://localhost/
2023-05-06T20:48:00.573-0400	Failed: cannot decode array into a primitive.D
2023-05-06T20:48:00.573-0400	0 document(s) imported successfully. 0 document(s) failed to import.
(PythonData) Crisaldry@MacBook-Air Resources % mongoimport --db uk_food --collection establishments --file establishments.json

2023-05-06T20:49:49.308-0400	connected to: mongodb://localhost/
2023-05-06T20:49:49.719-0400	Failed: cannot decode array into a primitive.D
2023-05-06T20:49:49.719-0400	0 document(s) imported successfully. 0 document(s) failed to import.
(PythonData) Crisaldry@MacBook-Air Resources % mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray

2023-05-06T20:49:56.105-0400	connected to: mongodb://localhost/
2023-05-06T20:49:57.788-0400	39779 document(s) imported successfully. 0 document(s) failed to import.
(PythonData) Crisaldry@MacBook-Air Resources % `

In [ ]:

# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created

db_names = mongo.list_database_names()
if 'uk_food' in db_names:
    print('The database exists')
else:
    print('The database does not exist')

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database

# list of collection names in the db
collection_names = db.list_collection_names()

# Print the list
print(collection_names)


In [ ]:

# review the collections in our new database
collection = db.get_collection('establishments')
print(collection)

In [ ]:
# review a document in the establishments collection
# review a document in the establishments collection
document = collection.find_one()

print(document)

In [ ]:
# assign the collection to a variable
collection = db['establishments']

collection

In [ ]:
collection

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data

# Create a dictionary for the new restaurant data
penang_flavours = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [ ]:

# Insert the new restaurant into the collection
result = collection.insert_one(penang_flavours)
print(f"Inserted new restaurant with ID {result.inserted_id}")

In [ ]:

# Check that the new restaurant was inserted
new_restaurant_fhrsid = 123456
result = collection.find_one({"FHRSID": new_restaurant_fhrsid})

if result:
    print("New restaurant was successfully inserted.")
    print(result)
else:
    print("Failed to insert new restaurant.")

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}

results = collection.find(query, projection)

for result in results:
    print(result)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Perform a query to find the document with the given FHRSID
query = {"FHRSID": new_restaurant_fhrsid}
result = collection.find_one(query)

if result:
    print("Restaurant with the specified FHRSID exists in the collection.")
else:
    print("Restaurant with the specified FHRSID does not exist in the collection.")


In [ ]:

# Update the new restaurant with the correct BusinessTypeID
correct_business_type_id = "7842"
filter_query = {"FHRSID": new_restaurant_fhrsid}
update_query = {"$set": {"BusinessTypeID": correct_business_type_id}}

result = collection.update_one(filter_query, update_query)

if result.matched_count == 1 and result.modified_count == 1:
    print("New restaurant was successfully updated.")
else:
    print("Failed to update new restaurant.")

In [ ]:
# Confirm that the new restaurant was updated
query = {"FHRSID": new_restaurant_fhrsid}
projection = {"_id": 0, "BusinessTypeID": 1}

result = collection.find_one(query, projection)

if result is not None:
    print("BusinessTypeID of Penang Flavours:", result['BusinessTypeID'])
else:
    print("Penang Flavours not found in database.")

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:

# Find how many documents have LocalAuthorityName as "Dover"

dover_query = {"LocalAuthorityName": "Dover"}

dover_count = collection.count_documents(dover_query)

print("Number of documents with LocalAuthorityName as Dover:", dover_count)


In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"

dover_query = {"LocalAuthorityName": "Dover"}

result = collection.delete_many(dover_query)

print(result.deleted_count, "documents deleted.")


In [ ]:
# Check if any remaining documents include Dover
dover_query = {"LocalAuthorityName": "Dover"}

dover_count = collection.count_documents(dover_query)

print("Number of documents with LocalAuthorityName as Dover:", dover_count)


In [ ]:

# Check that other documents remain with 'find_one'
dover_query = {"LocalAuthorityName": "Dover"}

result = collection.delete_many(dover_query)

print(result.deleted_count, "documents deleted.")

remaining_document = collection.find_one()

print("A document from the remaining collection:")
print(remaining_document)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
collection.update_many({}, [{'$set': {'geocode.longitude': {'$toDouble': '$geocode.longitude'}, 
                                           'geocode.latitude': {'$toDouble': '$geocode.latitude'}
                                          }
                                 }
                                ]
                           )

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Convert latitude and longitude to decimal numbers
filter_query_geo = {"geocode.latitude": {"$exists": True}, "geocode.longitude": {"$exists": True}}
update_query_geo = [
    {"$set": {"geocode.latitude": {"$toDouble": "$geocode.latitude"}}},
    {"$set": {"geocode.longitude": {"$toDouble": "$geocode.longitude"}}}
]
result_geo = collection.update_many(filter_query_geo, update_query_geo)

# Convert RatingValue to integer numbers
filter_query_rating = {"RatingValue": {"$exists": True}}
update_query_rating = {"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}
result_rating = collection.update_many(filter_query_rating, update_query_rating)


In [ ]:
# Query the documents after the updates
updated_documents = collection.find({})

# Print the latitude and longitude fields of the updated documents
for document in updated_documents:
    latitude = document["geocode"]["latitude"]
    longitude = document["geocode"]["longitude"]
    print("Latitude:", latitude)
    print("Longitude:", longitude)


In [ ]:
# Change the data type from String to Integer for RatingValue

# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
collection.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

# Change the data type from String to Integer for RatingValue
pipeline_2 = [
    {
        "$project": {
            "BusinessName": 1,
            "RatingValue": {
                "$toInt": "$RatingValue"
            },
            "AddressLine1": 1,
            "AddressLine2": 1,
            "AddressLine3": 1,
            "AddressLine4": 1,
            "PostCode": 1,
            "LocalAuthorityName": 1
        }
    }
]


In [ ]:
# Check that the coordinates and rating value are now numbers
# Query a document to check the data types
document = collection.find_one()

# Check the data types of the coordinates and RatingValue fields
latitude = document['geocode']['latitude']
longitude = document['geocode']['longitude']
rating_value = document['RatingValue']

print("Data types:")
print("Latitude:", type(latitude))
print("Longitude:", type(longitude))
print("RatingValue:", type(rating_value))
